# FUNCTIONS

In [ ]:
# !git clone https://github.com/Line233/one.git ./one
# !mv ./one/tool.py ./

In [ ]:
import torch
import random
import torchvision
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torch.utils.data as data
import torch.nn as nn
import tool
%matplotlib inline
import importlib
importlib.reload(tool)

In [ ]:
class mnist_data(data.Dataset):
    def __init__(self,x,y):
        self.x=x.float()
        self.y=y.long()
    def __len__(self):
        return len(self.x)
    def __getitem__(self,index):
        if self.y is not None:
            return self.x[index].reshape([-1]),self.y[index].long()
        else:
            return self.x[index].reshape([-1])

In [ ]:
class mnist_cov(nn.Module):
    def __init__(self,input_a,input_channel,device=torch.device('cpu')):
        super(mnist_cov,self).__init__()
        self.device=device
        self.inputa=input_a
        self.input_channel=input_channel
        self.conv1= nn.Conv2d(1,3,3,1,0)
        self.conv2=nn.Conv2d(3,6,3,2)
        self.conv3=nn.Conv2d(6,12,3,1)
        self.relu=nn.ReLU()
        self.out=nn.Sequential(
            nn.Linear(12,10),
            nn.Softmax(1)
        )
        self.lossF=nn.CrossEntropyLoss()
        pass
    def forward(self,input):
        input=input.reshape([-1,self.input_channel,self.inputa,self.inputa]).to(self.device)
        res= self.conv1(input)
        res=self.relu(res)
        res=self.conv2(res)
        res=self.relu(res)
        res=self.conv3(res)
        res=self.relu(res)
        res=res.reshape([-1,12])
        res=self.out(res)
        return res
    def reset_loss_record(self):
        self.__total_loss__=0.0
        self.__right_num__=0
        self.__total_num__=0
    def computeloss(self,x,y,res):
        loss=self.lossF(res,y)
        self.__total_loss__+=loss.tolist()*len(x)
        a=res.argmax(dim=1)
        self.__right_num__+=torch.eq(a,y).sum().tolist()
        self.__total_num__+=len(x)
        return loss
    def get_loss(self):
        return self.__total_loss__/self.__total_num__,self.__right_num__/self.__total_num__

# DATA

In [ ]:
train_data=torch.load('./data/train_hidden')
valid_data=torch.load('./data/valid_hidden')
test_data=torch.load('./data/test_hidden')

In [ ]:
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
simple_data=mnist_data(valid_data[0:100][0].reshape([-1,10,10]),valid_data[0:100][1])

# TRAIN

In [ ]:
model=mnist_cov(10,1)

In [ ]:
tool.TrainTool.train_epoch(model,train_data,valid_data,lr=0.001)